In [1]:
import math
import pandas as pd
from lenskit import topn
from sklearn.metrics import ndcg_score
from statistics import mean, variance
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.metrics.topn import ndcg
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.metrics.topn import ndcg
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
preprocessed_dataset_folder = "../preprocessed_dataset"
ratings_df = pd.read_csv(preprocessed_dataset_folder+"/ratings.csv") 
movies_df = pd.read_csv(preprocessed_dataset_folder+"/movies.csv", index_col="item")

In [3]:
def group_pipeline(groupsize):
    
    users_ratings = ratings_df.groupby(['user']).count()
    selected = users_ratings['rating'] > 200
    selected_users = users_ratings.loc[selected]
    random_selected = selected_users.sample(groupsize) # sample() returns a random row from the dataframe. The returned object is a dataframe with one row. If you pass a number as argument you specify to select more than one row.
    select_column_df = random_selected.reset_index()['user'] # reset_index() create a new index, and the user became a column. Then, we can filter using the column name
    selected_users = select_column_df.values # iloc select by index, since our dataframe only has one row we read it from the index 0
    print("Selected users: " + str(selected_users))

    train_df, test_df = train_test_split(ratings_df, test_size=0.2)

    user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
    recsys = Recommender.adapt(user_user)
    recsys.fit(train_df)

    test_df['predicted_rating'] = recsys.predict(test_df)

    test_df['relevant'] = test_df['rating'].apply(lambda x: 1 if x > 3 else 0)
    test_df['predicted_relevant'] = test_df['predicted_rating'].apply(lambda x: 1 if x > 3 else 0)

    train_df, test_df = train_test_split(ratings_df, test_size=0.2)

    user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
    recsys = Recommender.adapt(user_user)
    recsys.fit(train_df)
    
    group_list = []
    for i in selected_users:
        selected_movies = recsys.recommend(i, 10)
        group_list.append(selected_movies)
        
    group_df = pd.concat(group_list)
    group_df.drop_duplicates(subset=["item"], inplace=True)
    
    lst = []
    for i in selected_users:
        grp = group_df.copy()
        grp["user"] = i
        grp["pred"] = recsys.predict_for_user(i, grp["item"]).values
        lst.append(grp)
        
    final = pd.concat(lst)
    #Here I implement both least misery and additive aggregations strategies
    final.sort_values(by=["pred"], inplace=True, ascending=False)

    gbadd = final.groupby("item", as_index=False)["pred"].sum() #Additive strat
    gbleast = final.groupby("item", as_index=False)["pred"].min() #Least misery strat

    final_recommendationsAdd = gbadd.sort_values(by=["pred"], ascending=False).head(5)
    final_recommendationsAdd
    
    final_recommendationsleast = gbleast.sort_values(by=["pred"], ascending=False).head(5)
    final_recommendationsleast
        
    evaluateAdd = final_recommendationsAdd.copy()
    for i in selected_users:
        evaluateAdd[i] = recsys.predict_for_user(i, final_recommendationsAdd["item"]).values
    evaluateAdd
    # if scores cannot be predicted replace with 0
    evaluateAdd.fillna(0, inplace=True)
    recommended_ordering = pd.DataFrame(evaluateAdd[["item","pred"]])
    recommended_ordering
        
    evaluateleast = final_recommendationsleast.copy()
    for i in selected_users:
        evaluateleast[i] = recsys.predict_for_user(i, final_recommendationsleast["item"]).values
    evaluateleast
    # if scores cannot be predicted replace them with null value
    evaluateleast.fillna(0, inplace=True)
    recommended_orderingleast = pd.DataFrame(evaluateleast[["item","pred"]])
    recommended_orderingleast
    
    
    ndcg_lstAdd = []
    for i in selected_users:
        evaluateAdd.sort_values(by=[i], ascending=False, inplace=True)
        ev = pd.DataFrame(evaluateAdd[["item", i]])
        val = ndcg_score([ev.item], [recommended_ordering.item])
        ndcg_lstAdd.append(val)

    return mean(ndcg_lstAdd)
    
    
    ndcg_lstleast = []
    for i in selected_users:
        evaluateleast.sort_values(by=[i], ascending=False, inplace=True)
        ev = pd.DataFrame(evaluateleast[["item", i]])
        val = ndcg_score([ev.item], [recommended_orderingleast.item])
        ndcg_lstleast.append(val)

    return mean(ndcg_lstleast)

        

In [4]:
gsize2 = []
for i in range(100):
    gsize2.append(group_pipeline(4))

mean(gsize2)

Selected users: [ 28 294 509 555]


BLAS using multiple threads - can cause oversubscription
found 1 potential runtime problems - see https://boi.st/lkpy-perf


Selected users: [298 414 226 517]
Selected users: [294 292 514 200]
Selected users: [480 517 125 555]
Selected users: [469 509 596 288]
Selected users: [603 307 294  42]
Selected users: [368 573 483 599]
Selected users: [182 489 217 232]
Selected users: [600 202 274 391]
Selected users: [489 474 387 313]
Selected users: [160 177 202  57]
Selected users: [567 313 590 160]
Selected users: [ 89  68 288 483]
Selected users: [202 561 414 474]
Selected users: [275 307 103 387]
Selected users: [414 182  68 275]
Selected users: [599 573 292 226]
Selected users: [318 480  42 483]
Selected users: [606 219 226 200]
Selected users: [603 275 599 514]
Selected users: [232  57 514 606]
Selected users: [608  91 489 391]
Selected users: [480 160  64 438]
Selected users: [525 292 103 600]
Selected users: [573 307 226  89]
Selected users: [140 177  91 382]
Selected users: [274 610 477 448]
Selected users: [599 387 103 610]
Selected users: [561 307 391 226]
Selected users: [590 217 103 428]
Selected users

0.8004244882880661